# =============
# 《数据采集自检程序》1.9.5
## 一.基础信息检查
## 二.title、raw_title字段检查
## 三. keywords、terms字段查询
## 四. authors字段检查
## 五. src、sid、lang、venue、ts字段检查
## 六. issn、isbn、doi、pdf_src、paper_type、editor字段检查
## 七. year、volume、issue、page_start、page_end、page_str字段查询
# =============

In [ ]:
import numpy as np
import pandas as pd
import json
import re
import time
from collections import Counter

#### 以下两种为读取json文件的函数，运行一种无法读取，请尝试切换另一种

In [ ]:
data = pd.read_json('E:\.metadata\alrjournal_journals_urls_col.json',encoding='utf-8',lines=False)

In [40]:
data_0 = json.load(open('E:\\.metadata\\alrjournal_journals_urls_col.json',encoding = 'utf-8'))

data = pd.read_json((json.dumps(data_0)))

In [41]:
data.head()

,_id,ts,url,title,raw_title,hash,pdf_src,lang,year,oa_type,...,abstract,reference,corresponding_raw,emails,authors,issue,page_end,page_str,citation,orgs
0,{'$oid': '5f21366ff53d79186c04cf49'},{'$date': '2020-07-29T16:42:23.000+0000'},[https://www.alr-journal.org/articles/alr/full...,{'unknown': 'Satellite image analysis reveals ...,"{'unknown': '<h2 class=""title"">Satellite image...",siarcisbavpbvdtl3y,[https://www.alr-journal.org/articles/alr/pdf/...,en,2020,free,...,{'en': '<p>Seagrass meadows are fragile ecosys...,"[{'unknown': '<li> <a name=""R1...",\n* Corresponding author: nguyenxuanvi@gmail.c...,nguyenxuanvi@gmail.com,"[{'pos': 0, 'name': {'unknown': 'Trong-Thach V...",NaN,NaN,NaN,NaN,NaN
1,{'$oid': '5f213675f53d79186c04cf4a'},{'$date': '2020-07-29T16:42:29.000+0000'},[https://www.alr-journal.org/articles/alr/abs/...,{'unknown': 'Pattern of movements within a hom...,"{'unknown': '<h2 class=""title"">Pattern of move...",pomwahritci(sbteggel,[https://www.alr-journal.org/articles/alr/pdf/...,en,2015,free,...,{'en': '<p>This study determined the movements...,"[{'unknown': '<meta name=""citation_reference"" ...",a Corresponding author: eric.clua@gmail.com,eric.clua@gmail.com,"[{'pos': 0, 'name': {'unknown': 'Eric Clua'}, ...",1,58.0,53-58,NaN,NaN
2,{'$oid': '5f213676f53d79186c04cf4b'},{'$date': '2020-07-29T16:42:30.000+0000'},[https://www.alr-journal.org/articles/alr/abs/...,{'unknown': 'Variation of antioxidant/detoxifi...,"{'unknown': '<h2 class=""title"">Variation of an...",voaeairtbitgcpt,[https://www.alr-journal.org/articles/alr/pdf/...,en,2015,free,...,{'en': '<p>Responses of four toxicological ind...,"[{'unknown': '<meta name=""citation_reference"" ...",a Corresponding author: jerrylee200224@126.com,jerrylee200224@126.com,"[{'pos': 0, 'name': {'unknown': 'Lei Li'}, 'or...",1,51.0,45-51,NaN,NaN
3,{'$oid': '5f213677f53d79186c04cf4c'},{'$date': '2020-07-29T16:42:21.000+0000'},[https://www.alr-journal.org/articles/alr/full...,{'unknown': 'First record of Pampus minor (Act...,"{'unknown': '<h2 class=""title"">First record of...",fropm(psftcwowc,[https://www.alr-journal.org/articles/alr/pdf/...,en,2020,free,...,{'en': '<p><i>Pampus</i> fishes (Perciformes: ...,"[{'unknown': '<li> <a name=""R1...",\n* Corresponding author: lshlin@tio.org.cn; l...,lshlin@tio.org.cn;liyuan@tio.org.cn,"[{'pos': 0, 'name': {'unknown': 'Cheng Liu'}, ...",NaN,NaN,NaN,NaN,NaN
4,{'$oid': '5f21367bf53d79186c04cf4d'},{'$date': '2020-07-29T16:42:23.000+0000'},[https://www.alr-journal.org/articles/alr/abs/...,{'unknown': 'Sex-based spatial segregation of ...,"{'unknown': '<h2 class=""title"">Sex-based spati...",sssoabsclitncgl,[https://www.alr-journal.org/articles/alr/pdf/...,en,2013,free,...,{'en': '<p>Conservation of threatened large sh...,"[{'unknown': '<meta name=""citation_reference"" ...",a Corresponding author: jwerry@oc-research.com,jwerry@oc-research.com,"[{'pos': 0, 'name': {'unknown': 'Jonathan Mark...",4,288.0,281-288,"[{'unknown': '<article data-dkey=""10.1051/alr/...",NaN


# =============
# 一. 基础信息查询
# =============

## 1. 数据总数

In [37]:
print('数据总量：',len(data))

数据总量： 1271


## 2. 数据包含字段

In [ ]:
print('数据包含的字段名称如下')
data_str_len = len(data.columns)
for i in range(data_str_len):    
    print(data.columns[i])
print('数据字段数目：' + str(data_str_len) + '\n')

importion_str = ['url','title','raw_title','year','src','sid','ts']
importion_str_tag = 0
for i in range(len(importion_str)):
    if importion_str[i] not in data.columns:
        print('--------------------')
        print('异常，可能缺少必填字段' + importion_str[i])
    else:
        importion_str_tag = 1
if importion_str_tag == 0:
    print('正常，不缺少任何必填字段！')

## 3. 检查是否存在重复数据

In [ ]:
urllist = []
for url in data['url']:
    urllist.append(url[0])

url_list = list(set(urllist))

if len(url_list) == len(urllist):
    print('正常，未发现重复数据！')
else:
    print('异常，可能存在重复数据')
    print('【重复数据量 / 数据总量】  <==>  【',len(urllist)-len(url_list),'/',len(data),'】\n')

    #展现重复元素和重复次数
    repeation_tag = dict(Counter(urllist))
    for key,value in repeation_tag.items():
        if value>1:
            print('url：',key,'\t出现次数：',value)

## 4. 检查url的数据类型（url-List[String]）

In [ ]:
if 'url' in data.columns:
    have_url = data[data['url'].notnull()]
    url_type = []
    url_type_warn = 0
    for i in range(len(have_url)):
        url_type.append(str(isinstance(have_url.iloc[i]['url'],list)))
        if not isinstance(have_url.iloc[i]['url'],list):
            url_type_warn = url_type_warn + 1
    if 'False' not in url_type:
        print('正常，url数据类型正确')
    else:
        print('异常，url数据类型可能存在错误')
        print('【url数据类型可能存在错误的数据量 / 存在url的数据总量】  <==>  【',url_type_warn,'/',len(have_url),'】\n')
        for i in range(len(have_url)):
            if not isinstance(have_url.iloc[i]['url'],list):
                print('出现问题的url数据类型：',type(have_url.iloc[i]['url']))
else:
    print('异常，可能缺少必填字段url')

# =============
# 二. title, raw_title字段查询
# =============

## 1. 检查title和raw_title的数据类型（title-MultiLangString、raw_title-MultiLangHtmlString）

**(1) 检查 title的数据类型**

In [ ]:
if 'title' in data.columns:
    have_title = data[data['title'].notnull()]
    title_type = []
    title_type_warn = 0
    for i in range(len(have_title)):
        title_type.append(str(isinstance(have_title.iloc[i]['title'],dict)))
        if not isinstance(have_title.iloc[i]['title'],dict):
            title_type_warn = title_type_warn + 1
    if 'False' not in title_type:
        print('正常，title数据类型正确')
    else:
        print('异常，title数据类型可能存在错误')
        print('【title数据类型可能存在错误的数据量 / 存在title的数据总量】  <==>  【',title_type_warn,'/',len(have_title),'】\n')
        for i in range(len(have_title)):
            if not isinstance(have_title.iloc[i]['title'],dict):
                print('出现问题的title数据类型：',type(have_title.iloc[i]['title']))
                print('url：',have_title.iloc[i]['url'][0])
else:
    print('异常，可能缺少必填字段title')

**(2) 检查 raw_title的数据类型**

In [ ]:
if 'raw_title' in data.columns:
    have_raw_title = data[data['raw_title'].notnull()]
    raw_title_type = []
    raw_title_type_warn = 0

    for i in range(len(have_raw_title)):
        raw_title_type.append(str(isinstance(have_raw_title.iloc[i]['raw_title'],dict)))
        if not isinstance(have_raw_title.iloc[i]['raw_title'],dict):
            raw_title_type_warn = raw_title_type_warn + 1
        else:
            lang_list = list(have_raw_title.iloc[i]['raw_title'].keys())
            for t in range(len(lang_list)):
                if (have_raw_title.iloc[i]['raw_title'][lang_list[t]][0] != '<') or (have_raw_title.iloc[i]['raw_title'][lang_list[t]][-1] != '>') :
                    raw_title_type_warn = raw_title_type_warn + 1
                    raw_title_type.append('False')
    if 'False' not in raw_title_type:
        print('正常，raw_title数据类型正确')
    else:
        print('异常，raw_title数据类型可能存在错误')
        print('【raw_title数据类型可能存在错误的数据量 / 存在raw_title的数据总量】  <==>  【',raw_title_type_warn,'/',len(have_raw_title),'】\n')
        for i in range(len(have_raw_title)):
            if not isinstance(have_raw_title.iloc[i]['raw_title'],dict):
                print('出现问题的raw_title数据类型：',type(have_raw_title.iloc[i]['raw_title']))
                print('url：',have_raw_title.iloc[i]['url'][0],'\n')
            else:
                lang_list = list(have_raw_title.iloc[i]['raw_title'].keys())
                for t in range(len(lang_list)):
                    if (have_raw_title.iloc[i]['raw_title'][lang_list[t]][0] != '<') or (have_raw_title.iloc[i]['raw_title'][lang_list[t]][-1] != '>') :
                        print('raw_title可能非Html String类型：')
                        print(lang_list[t],': ',have_raw_title.iloc[i]['raw_title'][lang_list[t]])
                        print('url：',have_raw_title.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少必填字段raw_title')

## 2. 检查raw_title和title的数量、缺失情况

**(1) 检查 没有title字段，没有raw_title字段 情况**

In [ ]:
if ('title' in data.columns) and ('raw_title' in data.columns):
    no_title = data[data['title'].isnull()]
    no_title_no_raw_title = no_title[no_title['raw_title'].isnull()]

    if len(no_title_no_raw_title):
        print('异常，可能存在没有title，没有raw_title的情况')
        print('【没有title，没有raw_title的数据量 / 数据总量】  <==>  【',len(no_title_no_raw_title),'/',len(data),'】\n')
        print('请详细检查以下数据源：')
        for i in range(len(no_title_no_raw_title)):
            print('url：',no_title_no_raw_title.iloc[i]['url'][0])
    else:
        print('正常，不存在没有title，没有raw_title的情况')
else:
    if 'title' not in data.columns:
        print('异常，可能缺少必填字段title')
    if 'raw_title' not in data.columns:
        print('异常，可能缺少必填字段raw_title')

**(2) 检查 有title字段，没有raw_title字段 情况**

In [ ]:
if ('title' in data.columns) and ('raw_title' in data.columns):
    have_title = data[data['title'].notnull()]
    have_title_no_raw_title = have_title[have_title['raw_title'].isnull()]

    if len(have_title_no_raw_title):
        print('异常，可能存在有title，没有raw_title的情况\n')
        print('【有title，没有raw_title的数据量 / 数据总量】  <==>  【',len(have_title_no_raw_title),'/',len(data),'】\n')
        print('请详细检查以下数据源：')
        for i in range(len(have_title_no_raw_title)):
            print('url：',have_title_no_raw_title.iloc[i]['url'][0])
    else:
        print('正常，不存在有title，没有raw_title的情况')
else:
    if 'title' not in data.columns:
        print('异常，可能缺少必填字段title')
    if 'raw_title' not in data.columns:
        print('异常，可能缺少必填字段raw_title')

**(3) 检查 没有title字段，有raw_title字段 情况**

In [ ]:
if ('title' in data.columns) and ('raw_title' in data.columns):
    have_raw_title = data[data['raw_title'].notnull()]
    have_raw_title_no_title = have_raw_title[have_raw_title['title'].isnull()]

    if len(have_raw_title_no_title):
        print('异常，可能存在没有title，有raw_title的情况\n')
        print('【没有title，有raw_title的数据量 / 数据总量】  <==>  【',len(have_raw_title_no_title),'/',len(data),'】\n')
        print('请详细检查以下数据源：')
        for i in range(len(have_raw_title_no_title)):
            print('url：',have_raw_title_no_title.iloc[i]['url'][0])
    else:
        print('正常，不存在没有title，有raw_title的情况')
else:
    if 'title' not in data.columns:
        print('异常，可能缺少必填字段title')
    if 'raw_title' not in data.columns:
        print('异常，可能缺少必填字段raw_title')

## 3. 检查title和raw_title常见问题

**(1) 检查 title字段为空情况**

In [ ]:
if 'title' in data.columns:
    have_title = data[data['title'].notnull()]
    tag = 0
    for i in range(len(have_title)):
        lang_list = list(have_title.iloc[i]['raw_title'].keys())
        for t in range(len(lang_list)):
            title = have_title.iloc[i]['title'][lang_list[t]].strip()
            if title == '':
                if tag == 0:
                    tag = 1
                    print('异常，可能存在title字段为空的情况\n')
                    print('请详细检查以下数据源：')
                print('title: ',have_title.iloc[i]['title'])
                print('url：',have_title.iloc[i]['url'][0])
    if tag == 0:
        print('正常，未发现title字段为空的情况')
else:
    print('异常，可能缺少必填字段title')

**(2) 检查 raw_title字段为空情况**

In [ ]:
if 'raw_title' in data.columns:
    have_raw_title = data[data['raw_title'].notnull()]
    tag = 0
    for i in range(len(have_raw_title)):
        lang_list = list(have_raw_title.iloc[i]['raw_title'].keys())
        for t in range(len(lang_list)):
            raw_title = have_raw_title.iloc[i]['raw_title'][lang_list[t]].strip()
            if raw_title == '':
                if tag == 0:
                    tag = 1
                    print('异常，可能存在raw_title字段为空的情况\n')
                    print('请详细检查以下数据源：')
                print('raw_title: ',have_raw_title.iloc[i]['title'])
                print('url：', have_raw_title.iloc[i]['url'][0])
    if tag == 0:
        print('正常，未发现raw_title字段为空的情况')
else:
    print('异常，可能缺少必填字段raw_title')

**(3) 检查title字段内容是否存在常见问题**

In [ ]:
if 'title' in data.columns:
    have_title = data[data['title'].notnull()]
    tag = 0
    for i in range(len(have_title)):
        lang_list = list(have_title.iloc[i]['raw_title'].keys())
        for t in range(len(lang_list)):
            title = have_title.iloc[i]['title'][lang_list[t]]
            if title:
                if ('\n' in title) or ('\t' in title) or ('\r' in title) or ('  ' in title) or (not title[0].isalnum()) or (title[-1]==' '):
                    tag = tag + 1
                    if tag == 1:
                        print('异常，title字段中可能存在问题')
                    if ('\n' in title) or ('\t' in title) or ('\r' in title):
                        print('title：',have_title.iloc[i]['title'])
                        print('title可能存在转义字符')
                        print('url：',have_title.iloc[i]['url'][0],'\n')
                    if ('  ' in title) or (title[0] == ' ') or (title[-1] == ' '):
                        print('title：',have_title.iloc[i]['title'])
                        print('title可能存在多余空格')
                        print('url：',have_title.iloc[i]['url'][0],'\n')
                    if not title[0].isalnum():
                        if title[0] != ' ':
                            print('title：',have_title.iloc[i]['title'])
                            print('title中存在异常')
                            print('url：',have_title.iloc[i]['url'][0],'\n')
            else:
                tag = 1
                print('title：',have_title.iloc[i]['title'])
                print('title存在内容为空')
                print('url：',have_title.iloc[i]['url'][0],'\n')
    if tag == 0:
        print('正常，title字段未发现常见问题')
else:
    print('异常，可能缺少必填字段title')

# =============
# 二. abstract字段查询
# =============

## 1. 检查abstract数据类型（abstract-MultiLangHtmlString）

In [ ]:
if 'abstract' in data.columns:
    have_abstract = data[data['abstract'].notnull()]
    abstract_type = []
    abstract_type_warn = 0
    if 'abstract' in data.columns:
        for i in range(len(have_abstract)):
            abstract_type.append(str(isinstance(have_abstract.iloc[i]['abstract'],dict)))
            if not isinstance(have_abstract.iloc[i]['abstract'],dict):
                abstract_type_warn = abstract_type_warn + 1
        if 'False' not in abstract_type:
            print('正常，abstract数据类型正确')
        else:
            print('异常，abstract数据类型可能存在错误')
            print('【abstract数据类型可能存在错误的数据量 / 存在abstract的数据总量】  <==>  【',abstract_type_warn,'/',len(have_abstract),'】\n')
            for i in range(len(have_abstract)):
                if not isinstance(have_abstract.iloc[i]['abstract'],dict):
                    print('出现问题的abstract数据类型：',type(have_abstract.iloc[i]['abstract']))
                    print('url：',have_abstract.iloc[i]['url'][0])
else:
    print('异常，可能缺少必填字段abstract')

## 2. 检查abstract数据

**(1) 检查 abstract字段缺失的情况**

In [ ]:
if 'abstract' in data.columns:
    no_abstract = data[data['abstract'].isnull()]
    if len(no_abstract) == 0:
        print('正常，未发现abstract缺失现象')
    else:
        print('异常，可能存在abstract缺失现象')
        print('【abstract数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_abstract),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_abstract)):
            print('url：',no_abstract.iloc[i]['url'][0])
else:
    print('异常，可能缺少必填字段abstract')

**(2) 检查 abstract字段异常的情况**

In [ ]:
if 'abstract' in data.columns:
    have_abstract = data[data['abstract'].notnull()]
    tag = 0
    for i in range(len(have_abstract)):
        lang_list = list(have_abstract.iloc[i]['abstract'].keys())
        for t in range(len(lang_list)):
            abstract = have_abstract.iloc[i]['abstract'][lang_list[t]].strip()
            if len(abstract) != 0:
                if len(abstract) < 200 :
                    if tag == 0:
                        tag = 1
                        print('异常，abstract过短，可能存在问题')
                    print('abstract : ',have_abstract.iloc[i]['abstract'])
                    print('url：',have_abstract.iloc[i]['url'][0],'\n')
            else:
                print('异常，abstract字段可能为空')
                print('abstract : ',have_abstract.iloc[i]['abstract'])
                print('url：',have_abstract.iloc[i]['url'][0],'\n')
    if tag == 0:
        print('正常，未发现abstract异常情况')
                
else:
    print('异常，可能缺少必填字段abstract')

# =============
# 三. keywords、terms字段检查
# =============

## 1. 检查keywords字段

#### (1) 检查 keywords字段数据格式（keywords-MultiLangString）

In [ ]:
if 'keywords' in data.columns:
    have_keywords = data[data['keywords'].notnull()]
    keywords_type = []
    keywords_type_warn = 0
    for i in range(len(have_keywords)):
        keywords_type.append(str(isinstance(have_keywords.iloc[i]['keywords'],dict)))
        if not isinstance(have_keywords.iloc[i]['keywords'],dict):
            keywords_type_warn = keywords_type_warn + 1
    if 'False' not in keywords_type:
        print('正常，keywords数据类型正确')
    else:
        print('异常，keywords数据类型可能存在错误')
        print('【keywords数据类型可能存在错误的数据量 / 存在keywords的数据总量】  <==>  【',keywords_type_warn,'/',len(have_keywords),'】\n')
        for i in range(len(have_keywords)):
            if not isinstance(have_keywords.iloc[i]['keywords'],dict):
                print('出现问题的keywords数据类型：',type(have_keywords.iloc[i]['keywords']))
                print('url：',have_keywords.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段keywords，请详细检查数据源')

**(2) 检查 keywords字段缺失问题**

In [ ]:
if 'keywords' in data.columns:
    no_keywords = data[data['keywords'].isnull()]
    if len(no_keywords) == 0:
        print('正常，未发现keywords缺失现象')
    else:
        print('异常，可能存在keywords缺失现象')
        print('【keywords数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_keywords),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_keywords)):
            print('url：',no_keywords.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段keywords，请详细检查数据源')

**(3) 检查 keywords字段数据问题**

In [39]:
if 'keywords' in data.columns:
    have_keywords = data[data['keywords'].notnull()]
    tag = 0
    for i in range(len(have_keywords)):
        lang_list = list(have_keywords.iloc[i]['keywords'].keys())
        for t in range(len(lang_list)):
            keywords = str(have_keywords.iloc[i]['keywords'][lang_list[t]]).split(';')
            if len(keywords) == 1:
                if keywords[0] == 'nan':
                    continue
                else:
                    if tag == 0:
                        tag = 1
                        print('异常，keywords字段可能出现问题')
                        print('请详细检查以下keywords是否出现问题：\n')
                    print('keywords：',have_keywords.iloc[i]['keywords'])
                    print('url：',have_keywords.iloc[i]['url'][0],'\n')
            if (not have_keywords.iloc[i]['keywords'][lang_list[t]][0].isalnum()) or (not have_keywords.iloc[i]['keywords'][lang_list[t]][-1].isalnum()) or ('; ' in have_keywords.iloc[i]['keywords'][lang_list[t]]) or ('  ' in have_keywords.iloc[i]['keywords'][lang_list[t]]):
                if tag == 0:
                    tag = 1
                    print('异常，keywords字段可能出现问题')
                    print('请详细检查以下keywords是否出现问题：\n')
                print('keywords：',have_keywords.iloc[i]['keywords'])
                print('url：',have_keywords.iloc[i]['url'][0],'\n')
    if tag == 0:
        print('正常，未发现keywords存在问题')
else:
    print('异常，可能缺少字段keywords，请详细检查数据源')

异常，keywords字段可能出现问题
请详细检查以下keywords是否出现问题：

keywords： {'en': 'Aquaculture economic assessment;endemic snail;aquatic resource conservation;financial assessment;human pressure on snail population;Monte Carlo;stochastic modelling;Pomacea patula catemacensis (P. p. catemacensis)'}
url： https://www.alr-journal.org/articles/alr/full_html/2020/01/alr180056/alr180056.html 

keywords： {'en': 'Anadromous species;stock assessment;split-beam sonar;Oncorhynchus nerka;Oncorhynchus gorbuscha;Canada (British Columbia)'}
url： https://www.alr-journal.org/articles/alr/abs/1998/02/alr8277/alr8277.html 

keywords： {'en': 'length-frequency data;seasonal growth;reproductive strategy;upwelling ecosystem;Clupeidae;Engraulidae;Talcahuano (Chile)'}
url： https://www.alr-journal.org/articles/alr/abs/2001/02/alr1141/alr1141.html 

keywords： {'en': 'Combined quota and effort control;Bio-economic modelling;FLR (Fisheries Laboratory in language R)'}
url： https://www.alr-journal.org/articles/alr/abs/2008/03/alr028-08/a

## 2. 检查terms字段

#### (1) 检查 terms字段数据格式（terms-MultiLangString）

In [ ]:
if 'terms' in data.columns:
    have_terms = data[data['terms'].notnull()]
    terms_type = []
    terms_type_warn = 0
    for i in range(len(have_terms)):
        terms_type.append(str(isinstance(have_terms.iloc[i]['terms'],dict)))
        if not isinstance(have_terms.iloc[i]['terms'],dict):
            terms_type_warn = terms_type_warn + 1
    if 'False' not in terms_type:
        print('正常，terms数据类型正确')
    else:
        print('异常，terms数据类型可能存在错误')
        print('【terms数据类型可能存在错误的数据量 / 存在terms的数据总量】  <==>  【',terms_type_warn,'/',len(have_terms),'】\n')
        for i in range(len(have_terms)):
            if not isinstance(have_terms.iloc[i]['terms'],dict):
                print('出现问题的terms数据类型：',type(have_terms.iloc[i]['terms']))
                print('url：',have_terms.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段terms，请详细检查数据源')

**(2) 检查 terms字段缺失问题**

In [ ]:
if 'terms' in data.columns:
    no_terms = data[data['terms'].isnull()]
    if len(no_terms) == 0:
        print('正常，未发现terms缺失现象')
    else:
        print('异常，可能存在terms缺失现象')
        print('【terms数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_terms),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_terms)):
            print('url：',no_terms.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段terms，请详细检查数据源')

**(3) 检查 terms字段数据问题**

In [ ]:
if 'terms' in data.columns:
    have_terms = data[data['terms'].notnull()]
    tag = 0
    for i in range(len(have_terms)):
        lang_list = list(have_terms.iloc[i]['terms'].keys())
        for t in range(len(lang_list)):
            terms = str(have_terms.iloc[i]['terms'][lang_list[t]]).split(';')
            if len(terms) == 1:
                if terms[0] == 'nan':
                    continue
                else:
                    if tag == 0:
                        tag = 1
                        print('异常，terms字段可能出现问题')
                        print('请详细检查以下terms是否出现问题：\n')
                    print('terms：',have_terms.iloc[i]['terms'])
                    print('url：',have_terms.iloc[i]['url'][0],'\n')
            if (not have_terms.iloc[i]['terms'][lang_list[t]][0].isalnum()) or (not have_terms.iloc[i]['terms'][lang_list[t]][-1].isalnum()) or ('; ' in have_terms.iloc[i]['terms'][lang_list[t]]) or ('  ' in have_terms.iloc[i]['terms'][lang_list[t]]):
                if tag == 0:
                    tag = 1
                    print('异常，terms字段可能出现问题')
                    print('请详细检查以下terms是否出现问题：\n')
                print('terms：',have_terms.iloc[i]['terms'])
                print('url：',have_terms.iloc[i]['url'][0],'\n')
    if tag == 0:
        print('正常，未发现terms存在问题')
else:
    print('异常，可能缺少字段terms，请详细检查数据源')

# =============
# 四. authors字段检查
### 1. 检查authors字段缺失情况
### 2. 检查name字段
### 3. 检查org字段
### 4. 检查email字段
# =============

## 1. 检查authors字段缺失情况

In [ ]:
if 'authors' in data.columns:
    no_authors = data[data['authors'].isnull()]
    if len(no_authors) == 0:
        print('正常，未发现authors字段缺失情况')
    else:
        print('异常，可能存在authors缺失现象')
        print('【authors数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_authors),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_authors)):
            print('url：',no_authors.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

## 2. 检查pos字段

**(1) 检查 pos字段数据是否缺失**

In [ ]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                author = have_authors.iloc[i]['authors'][n]
                if 'pos' in author:
                    tag = 0
                else:
                    tag = 1
                    break
    if tag == 0:
        print('正常，未发现authors字段中pos缺失现象')
    else:
        print('异常，authors字段中可能缺少必填字段pos')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

**(2) 检查 pos字段数据类型（pos-Int32）**

In [ ]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                author = have_authors.iloc[i]['authors'][n]
                if 'pos' in author:
                    pos = author['pos']
                    if not isinstance(pos,int):
                        tag = 1
                        break
    if tag == 0:
        print('正常，未发现authors字段中pos数据格式问题')
    else:
        print('异常，authors字段中pos字段格式可能出现错误')
        print('要求格式为Int32，实际格式为',type(pos))
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

**(3) 检查 pos字段数据问题**

In [ ]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                author = have_authors.iloc[i]['authors'][n]
                if 'pos' in author:
                    pos = author['pos']
                    if isinstance(pos,int):
                        if pos != n:
                            tag = 1
                            break
    if tag == 0:
        print('正常，未发现authors字段中pos数据问题')
    else:
        print('异常，authors字段中pos字段数据可能出现错误')
        print('pos字段应该从 0 开始')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

## 3. 检查name字段

**(1) 检查 name字段数据问题**

In [ ]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                authors = have_authors.iloc[i]['authors'][n]
                if 'name' in authors:
                    namestr = authors['name']
                    lang_list = list(namestr.keys())
                    for t in range(len(lang_list)):
                        if namestr[lang_list[t]] :
                            if ('\n' in namestr[lang_list[t]]) or ('\t' in namestr[lang_list[t]]) or ('\r' in namestr[lang_list[t]]):
                                tag = 1
                            if (len(namestr[lang_list[t]]) <= 1) or (len(namestr[lang_list[t]]) > 30) or (' and ' in namestr[lang_list[t]]):
                                tag = 1
                            if ('(' in namestr[lang_list[t]]) or (not namestr[lang_list[t]][0].isalpha()) or (not namestr[lang_list[t]][-1].isalpha()):
                                if (namestr[lang_list[t]][0] == ' ') or (namestr[lang_list[t]][-1] == ' '):
                                    tag = 1
                                elif namestr[lang_list[t]][-1] != '.':
                                    tag = 1
                        else:
                            tag = 1
    if tag == 0:
        print('正常，未发现name存在问题')
    else:
        print('异常，name字段可能出现问题')
        print('请详细检查以下name是否出现问题：\n')
    
        for i in range(len(have_authors)):
            if len(have_authors.iloc[i]['authors']) != 0:
                for n in range(len(have_authors.iloc[i]['authors'])):
                    authors = have_authors.iloc[i]['authors'][n]
                    if 'name' in authors:
                        namestr = authors['name']
                        lang_list = list(namestr.keys())
                        for t in range(len(lang_list)):
                            if namestr[lang_list[t]] :
                                if ('\n' in namestr[lang_list[t]]) or ('\t' in namestr[lang_list[t]]) or ('\r' in namestr[lang_list[t]]):
                                    print('name中可能存在转义字符 ntr')
                                    for t in range(len(lang_list)):
                                        print(lang_list[t],' : ',namestr[lang_list[t]])
                                    print('url：',have_authors.iloc[i]['url'][0],'\n')
                                if (len(namestr[lang_list[t]]) <= 1) or (len(namestr[lang_list[t]]) > 30) or (' and ' in namestr[lang_list[t]]):
                                    print('name太短或太长，可能有问题')
                                    for t in range(len(lang_list)):
                                        print(lang_list[t],' : ',namestr[lang_list[t]])
                                    print('url：',have_authors.iloc[i]['url'][0],'\n')
                                if ('(' in namestr[lang_list[t]]) or (not namestr[lang_list[t]][0].isalpha()) or (not namestr[lang_list[t]][-1].isalpha()):
                                    if namestr[lang_list[t]].strip() == '':
                                        print('name可能为空')
                                        for t in range(len(lang_list)):
                                            print(lang_list[t],' : ',namestr[lang_list[t]])
                                        print('url：',have_authors.iloc[i]['url'][0],'\n')
                                    elif (namestr[lang_list[t]][0] == ' ') or (namestr[lang_list[t]][-1] == ' '):
                                        print('name两侧可能存在空格')
                                        for t in range(len(lang_list)):
                                            print(lang_list[t],' : ',namestr[lang_list[t]])
                                        print('url：',have_authors.iloc[i]['url'][0],'\n')
                                    elif namestr[lang_list[t]][-1] != '.':
                                        print('name中可能存在异常')
                                        for t in range(len(lang_list)):
                                            print(lang_list[t],' : ',namestr[lang_list[t]])
                                        print('url：',have_authors.iloc[i]['url'][0],'\n')
                            else:
                                    print('name为空')
                                    for t in range(len(lang_list)):
                                        print(lang_list[t],' : ',namestr[lang_list[t]])
                                    print('url：',have_authors.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

## 4. 检查作者机构

**(1) 检查作者机构缺失问题**

In [ ]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            orgs=have_authors.iloc[i].get('orgs')
            for n in range(len(have_authors.iloc[i]['authors'])):
                authors = have_authors.iloc[i]['authors'][n]
                if 'org' not in authors and orgs==None:
                    tag = 1
    if tag == 0:
        print('正常，未发现作者机构缺失问题')
    else:
        print('异常，可能缺失作者机构')
        for i in range(len(have_authors)):
            if len(have_authors.iloc[i]['authors']) != 0:
                orgs=have_authors.iloc[i].get('orgs')
                for n in range(len(have_authors.iloc[i]['authors'])):
                    authors = have_authors.iloc[i]['authors'][n]
                    if 'org' not in authors and orgs==None:
                        print('name：',authors['name'])
                        print('url：',have_authors.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

**(2) 检查 org字段数据问题**

In [42]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                authors = have_authors.iloc[i]['authors'][n]
                if 'org' in authors:
                    orglist = authors['org']
                    if len(orglist) != 0:
                        for t in range(len(orglist)):
                            orgdic = orglist[t]
                            lang_list = list(orgdic.keys())
                            for q in range(len(lang_list)):
                                lang = lang_list[q]
                                if len(orglist[t]) != 0:
                                    if ('\n' in orglist[t][lang]) or ('\t' in orglist[t][lang]) or ('\r' in orglist[t][lang]) or ('  ' in orglist[t][lang]):
                                        tag = 1
                                    try:
                                        if (not orglist[t][lang][0].isalnum()) or (not orglist[t][lang][-1].isalnum()):
                                            if orglist[t][lang][-1] != '.' and orglist[t][lang][-1] != ')':
                                                tag = 1
                                    except Exception:
                                        print(have_authors.iloc[i]['url'][0])
                                    if '@' in orglist[t][lang]:
                                        tag = 1
                                else:
                                    tag = 1
                    else:
                        tag = 1

    if tag == 0:
        print('正常，未发现org存在问题')
    else:
        print('异常，org字段可能出现问题')
        print('请详细检查以下org是否出现问题：\n')
    
        for i in range(len(have_authors)):
            if len(have_authors.iloc[i]['authors']) != 0:
                for n in range(len(have_authors.iloc[i]['authors'])):
                    authors = have_authors.iloc[i]['authors'][n]
                    if 'org' in authors:
                        orglist = authors['org']
                        if len(orglist) != 0:
                            for t in range(len(orglist)):
                                orgdic = orglist[t]
                                lang_list = list(orgdic.keys())
                                for q in range(len(lang_list)):
                                    lang = lang_list[q]
                                    if len(orglist[t]) != 0:
                                        if ('\n' in orglist[t][lang]) or ('\t' in orglist[t][lang]) or ('\r' in orglist[t][lang]):
                                            print('org中可能存在转义字符')
                                            print('org：',orglist[t])
                                            print('url：',have_auAthors.iloc[i]['url'][0],'\n')
                                        if (not orglist[t][lang][0].isalnum()) or (not orglist[t][lang][-1].isalnum()):
                                            if orglist[t][lang][-1] != '.' and orglist[t][lang][-1] != ')':
                                                print('org中可能存在异常')
                                                print('org：',orglist[t])
                                                print('url：',have_authors.iloc[i]['url'][0],'\n')
                                        if '@' in orglist[t][lang]:
                                            print('org中可能存在异常')
                                            print('org：',orglist[t])
                                            print('url：',have_authors.iloc[i]['url'][0],'\n')
                                        if '  ' in orglist[t][lang]:
                                            print('org中可能存在多余空格')
                                            print('org：',orglist[t])
                                            print('url：',have_authors.iloc[i]['url'][0],'\n')
                                    else:
                                        print('org中可能为空')
                                        print('org：',orglist)
                                        print('url：',have_authors.iloc[i]['url'][0],'\n')
                        else:
                            print('org中可能为空')
                            print('org：',orglist)
                            print('url：',have_authors.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

正常，未发现org存在问题


## 5. 检查email字段

**(1) 检查 email字段数据问题**

In [ ]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                authors = have_authors.iloc[i]['authors'][n]
                if 'email' in authors:
                    emailstr = authors['email']
                    if emailstr :
                        if ('\n' in emailstr) or ('\t' in emailstr) or ('\r' in emailstr):
                            tag = 1
                        if (' ' in emailstr) or ('@' not in emailstr):
                            tag = 1
                        if ('(' in emailstr) or (not emailstr[0].isalpha()) or (not emailstr[-1].isalpha()):
                            if (emailstr[0] == ' ') or (emailstr[-1] == ' '):
                                tag = 1
                    else:
                        tag = 1
    if tag == 0:
        print('正常，未发现email存在问题')
    else:
        print('异常，email字段可能出现问题')
        print('请详细检查以下email是否出现问题：\n')
    
        for i in range(len(have_authors)):
            if len(have_authors.iloc[i]['authors']) != 0:
                for n in range(len(have_authors.iloc[i]['authors'])):
                    authors = have_authors.iloc[i]['authors'][n]
                    if 'email' in authors:
                        emailstr = authors['email']
                        if emailstr :
                            if ('\n' in emailstr) or ('\t' in emailstr) or ('\r' in emailstr):
                                print('email中可能存在转义字符 ntr')
                                print('email：',emailstr)
                                print('url：',have_authors.iloc[i]['url'][0],'\n')
                            if (' ' in emailstr) or ('@' not in emailstr):
                                print('email中可能存在异常')
                                print('email：',emailstr)
                                print('url：',have_authors.iloc[i]['url'][0],'\n')
                            if ('(' in emailstr) or (not emailstr[0].isalnum()) or (not emailstr[-1].isalpha()):
                                if emailstr.strip() == '':
                                    print('email可能为空')
                                    print('email：',emailstr)
                                    print('url：',have_authors.iloc[i]['url'][0],'\n')
                                elif (emailstr[0] == ' ') or (emailstr[-1] == ' '):
                                    print('email两侧可能存在空格')
                                    print('email：',emailstr)
                                    print('url：',have_authors.iloc[i]['url'][0],'\n')
                        else:
                                print('email为空')
                                print('email：',emailstr)
                                print('url：',have_authors.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

## 6.检查homepage字段

### (1)检查homepage数据类型

In [ ]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                author = have_authors.iloc[i]['authors'][n]
                if 'homepage' in author:
                    homepage = author['homepage']
                    if not isinstance(homepage,str):
                        tag = 1
                        print("homepage类型:",type(homepage),have_authors.iloc[i]['url'][0])
    if tag == 0:
        print('正常，未发现authors字段中homepage类型有问题')
    else:
        print('异常，authors字段中homepage字段数据类型出现错误,应为String类型')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

### (2)检查homepage数据缺失

In [ ]:
if 'authors' in data.columns:
    have_authors = data[data['authors'].notnull()]
    tag = 0
    for i in range(len(have_authors)):
        if len(have_authors.iloc[i]['authors']) != 0:
            for n in range(len(have_authors.iloc[i]['authors'])):
                authors = have_authors.iloc[i]['authors'][n]
                if 'homepage' not in authors:
                    tag = 1
    if tag == 0:
        print('正常，未发现homepage缺失问题')
    else:
        print('异常，可能缺失homepage')
        for i in range(len(have_authors)):
            if len(have_authors.iloc[i]['authors']) != 0:
                for n in range(len(have_authors.iloc[i]['authors'])):
                    authors = have_authors.iloc[i]['authors'][n]
                    if 'homepage' not in authors:
                        print('name：',authors['name'])
                        print('url：',have_authors.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段authors，请详细检查数据源')

# =============
# 五. reference、citation字段查询
# =============

## 1. 检查reference字段

#### (1) 检查 reference字段数据格式（reference-List[MultiLangHtmlString]）

In [ ]:
if 'reference' in data.columns:
    have_reference = data[data['reference'].notnull()]
    reference_type = []
    reference_type_warn = 0

    for i in range(len(have_reference)):
        reference_type.append(str(isinstance(have_reference.iloc[i]['reference'],list)))
        if not isinstance(have_reference.iloc[i]['reference'],list):
            reference_type_warn = reference_type_warn + 1
        else:
            paper_reference = have_reference.iloc[i]['reference']
            for n in range(len(paper_reference)):
                reference = paper_reference[n]
                lang_list = list(reference.keys())
                for t in range(len(lang_list)):
                    if (reference[lang_list[t]] != '<') or (reference[lang_list[t]] != '>') :
                        reference_type_warn = reference_type_warn + 1
    if 'False' not in reference_type:
        print('正常，reference数据类型正确')
    else:
        print('异常，reference数据类型可能存在错误')
        print('【reference数据类型可能存在错误的数据量 / 存在reference的数据总量】  <==>  【',reference_type_warn,'/',len(have_reference),'】\n')
        for i in range(len(have_reference)):
            if not isinstance(have_reference.iloc[i]['reference'],str):
                print('出现问题的reference数据类型：',type(have_reference.iloc[i]['reference']))
                print('url：',have_reference.iloc[i]['url'][0],'\n')
            else:
                if (have_reference.iloc[i]['reference'][0][lang] != '<') or (have_reference.iloc[i]['reference'][-1][lang] != '>') :
                    print('reference可能非Html String类型：')
                    print('url：',have_reference.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段reference，请详细检查数据源')

**(2) 检查 reference字段数据缺失**

In [ ]:
if 'reference' in data.columns:
    no_reference = data[data['reference'].isnull()]
    if len(no_reference) == 0:
        print('正常，未发现reference字段缺失情况')
    else:
        print('异常，可能存在reference缺失现象')
        print('【reference数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_reference),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_reference)):
            print('url：',no_reference.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段reference，请详细检查数据源')

## 2. 检查citation字段

#### (1) 检查 citation字段数据格式（citation-List[MultiLangHtmlString]）

In [ ]:
if 'citation' in data.columns:
    have_citation = data[data['citation'].notnull()]
    citation_type = []
    citation_type_warn = 0

    for i in range(len(have_citation)):
        citation_type.append(str(isinstance(have_citation.iloc[i]['citation'],list)))
        if not isinstance(have_citation.iloc[i]['citation'],list):
            citation_type_warn = citation_type_warn + 1
        else:
            paper_citation = have_citation.iloc[i]['citation']
            for n in range(len(paper_citation)):
                citation = paper_citation[n]
                lang_list = list(citation.keys())
                for t in range(len(lang_list)):
                    if (citation[lang_list[t]] != '<') or (citation[lang_list[t]] != '>') :
                        citation_type_warn = citation_type_warn + 1
    if 'False' not in citation_type:
        print('正常，citation数据类型正确')
    else:
        print('异常，citation数据类型可能存在错误')
        print('【citation数据类型可能存在错误的数据量 / 存在citation的数据总量】  <==>  【',citation_type_warn,'/',len(have_citation),'】\n')
        for i in range(len(have_citation)):
            if not isinstance(have_citation.iloc[i]['citation'],str):
                print('出现问题的citation数据类型：',type(have_citation.iloc[i]['citation']))
                print('url：',have_citation.iloc[i]['url'][0],'\n')
            else:
                if (have_citation.iloc[i]['citation'][0][lang] != '<') or (have_citation.iloc[i]['citation'][-1][lang] != '>') :
                    print('citation可能非Html String类型：')
                    print('url：',have_citation.iloc[i]['url'][0],'\n')
else:
    print('异常，可能缺少字段citation，请详细检查数据源')

**(2) 检查 citation字段数据缺失**

In [ ]:
if 'citation' in data.columns:
    no_citation = data[data['citation'].isnull()]
    if len(no_citation) == 0:
        print('正常，未发现citation字段缺失情况')
    else:
        print('异常，可能存在citation缺失现象')
        print('【citation数据类型可能缺失的数据量 / 数据总量】  <==>  【',len(no_citation),'/',len(data),'】')
        print('请从以下随机抽样检查，查看是否存在抓取遗漏：\n')
        for i in range(len(no_citation)):
            print('url：',no_citation.iloc[i]['url'][0])
else:
    print('异常，可能缺少字段citation，请详细检查数据源')

# =============
# 五. src、sid、venue、ts字段检查
# =============

## 1. 检查src字段

**(1) 检查 src字段数据类型**

In [ ]:
if 'src' in data.columns:
    have_src = data[data['src'].notnull()]
    src_type = []
    for i in range(len(have_src)):
        src_type.append(str(isinstance(have_src.iloc[i]['src'],str)))
    if 'False' not in src_type:
        print('正常，src数据类型正确')
    else:
        print('异常，src数据类型可能存在错误\n')
        for i in range(len(have_src)):
            src = have_src.iloc[i]['src']
            if not isinstance(src,str):
                print('src的数据类型为：',type(src))
                print('url：',have_src.iloc[i]['url'][0])
else:
    print('异常，可能缺少必填字段src')

**(2) 检查 src字段数据问题**

In [ ]:
if 'src' in data.columns:
    have_src = data[data['src'].notnull()]
    src = have_src.iloc[0]['src']
    print('请详细检查，src字段应为 数据源名称')
    print('src：',src)
else:
    print('异常，可能缺少必填字段src')

## 2. 检查sid字段

**(1) 检查 sid字段数据重复**

In [ ]:
sidlist = []
for sid in data['sid']:
    sidlist.append(sid)

sid_list = list(set(sidlist))

if len(sid_list) == len(sidlist):
    print('正常，未发现重复数据！')
else:
    print('异常，可能存在重复数据')
    print('【重复数据量 / 数据总量】  <==>  【',len(sidlist)-len(sid_list),'/',len(data),'】\n')

    #展现重复元素和重复次数
    repeation_tag = dict(Counter(sidlist))
    for key,value in repeation_tag.items():
        if value>1:
            print('sid：',key,'\t出现次数：',value)

## 3. 检查venue字段

**(1) 检查 venue字段数据类型**

In [ ]:
if 'venue' in data.columns:
    have_venue = data[data['venue'].notnull()]
    venue_type = []
    venue_type.append(str(isinstance(have_venue.iloc[0]['venue'],dict)))
    venue_type.append(str(isinstance(have_venue.iloc[0]['venue']['name'],dict)))
    venue_type.append(str(isinstance(have_venue.iloc[0]['venue']['type'],int)))
    venue_type.append(str(isinstance(have_venue.iloc[0]['venue']['sid'],str)))
    if 'False' not in venue_type:
        print('正常，venue数据类型正确')
    else:
        print('异常，venue数据类型可能存在错误\n')
        venue = have_venue.iloc[0]['venue']
        venue_name = have_venue.iloc[0]['venue']['name']
        venue_type = have_venue.iloc[0]['venue']['type']
        venue_sid = have_venue.iloc[0]['venue']['sid']
        if (not isinstance(venue,dict)):
            print('venue的数据类型应为：dict')
            print('venue的数据类型为：',type(venue))
        if (not isinstance(venue_name,dict)):
            print('venue_name的数据类型应为：dict')
            print('venue_name的数据类型为：',type(venue_name),'\n')
        if (not isinstance(venue_type,int)):
            print('venue_type的数据类型应为：Int')
            print('venue_type的数据类型为：',type(venue_type),'\n')
        if (not isinstance(venue_sid,str)):
            print('venue_sid的数据类型应为：String')
            print('venue_sid的数据类型为：',type(venue_sid),'\n')
else:
    print('异常，可能缺少必填字段venue')

**(2) 检查 venue字段数据重复**

In [ ]:
venue_name = []
venue_type = []
venue_sid = []
for each in data['venue']:
    venue_name.append(each['name'])
    venue_type.append(each['type'])
    venue_sid.append(each['sid'])

#列表推导法
name_list = []
for one in venue_name:
    if one not in name_list:
        print(one)
        name_list.append(one)
if len(name_list) == 1:
    print('正常，未发现venue_name字段重复')
else:
    print('venue中的name不止一个，请重新检查：',name_list)
if len(set(venue_type)) == 1:
    print('正常，未发现venue_type字段重复')
else:
    print('venue中的type不止一个，请重新检查：',set(venue_type))
if len(set(venue_sid)) == 1:
    print('正常，未发现venue_sid字段重复')
else:
    print('venue中的sid不止一个，请重新检查：',set(venue_sid))

**(3) 检查 venue字段数据问题**

In [ ]:
if 'venue' in data.columns:
    have_venue = data[data['venue'].notnull()]
    venue = have_venue.iloc[0]['venue']
    venue_name = have_venue.iloc[0]['venue']['name']
    venue_type = have_venue.iloc[0]['venue']['type']
    venue_sid = have_venue.iloc[0]['venue']['sid']
    print('请详细检查，以下venue字段是否符合规定\n')
    print('venue_name应为：论文中的期刊会议书籍名称')
    print('venue_name为：',venue_name,'\n')
    print('venue_type应为：在venue表中定义的类别')
    print('venue_type为：',venue_type,'\n')
    print('venue_sid应为：在数据源中的Source id')
    print('venue_sid为：',venue_sid,'\n')
else:
    print('异常，可能缺少必填字段venue')

## 5. 检查ts字段

**(1) 检查 ts字段数据类型**

In [ ]:
if 'ts' in data.columns:
    if '$date' in data.iloc[0]['ts'] :
        ts = data.iloc[0]['ts']['$date']
        if 'T' in ts:
            ts = ts.split('.')[0].replace('T',' ')
            try:
                time.strptime(ts,"%Y-%m-%d %H:%M:%S")
                tag = 0
            except:
                tag = 1
        else:
            tag = 1
        if tag == 0:
            print('正常，字段ts的数据格式可能正确')
        else:
            print('异常，字段ts的数据格式可能出现问题')
        print(data.iloc[0]['ts']['$date'])
    else:
        print('异常，字段ts的数据格式可能出现问题')
else:
    print('异常，可能缺少必填字段ts')